### Fonction qui implémente la résolution des sous-problèmes

Les sous-problèmes s'écrivent ainsi :

max $w_c - \sum_{i \in V} \pi_i \sum_{j : (i,j) \in A} x_{ij}$

tel que :
- $\sum_{(i,j) \in A} x_{ij} \le L$,
- $\sum_{i : (i,j) \in A} x_{ij} = 1$,
- $\sum_{j : (i,j) \in A} x_{ij} = 1$,
- $x_{ij} \in \{0,1\}$.

In [6]:
function subproblem(l,π,node)
    SP=Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    set_optimizer_attribute(SP, "OutputFlag", 0)
    
    Vl = Vector{Int}()
    for i in (l:nv(new_inst.graph))
        dli = dijkstra(new_inst.edge_weight,l)[i]
        dil = dijkstra(new_inst.edge_weight,i)[l]
        if ((dli + dil <= new_inst.max_cycle_length) && !(i in Vl))
            push!(Vl,i)
        end
    end
    
    Al = zeros((16,16))
    for i in 1:nv(new_inst.graph)
        for j in 1:nv(new_inst.graph)
            dli = dijkstra(new_inst.edge_weight,l)[i]
            djl = dijkstra(new_inst.edge_weight,j)[l]
            if (i in Vl ) && (j in Vl) && (dli + djl + 1 <= new_inst.max_cycle_length)
                Al[i,j] = 1
            end
        end
    end
    Al[l,l]=0 # pas d'arc sur le même noeud
    
    # Variable
    @variable(SP,x[i in 1:nv(new_inst.graph),j in 1:nv(new_inst.graph)],Bin)
    
    # Contraintes
    # Le cycle doit être de longueur maximale L
    #@constraint(SP,sum(x[i,j] for i in l:nv(new_inst.graph),j in l:nv(new_inst.graph) if (new_inst.edge_weight[i,j]==1.0))<=new_inst.max_cycle_length)
    @constraint(SP,sum(x[i,j] for i in Vl, j in Vl if (Al[i,j]==1))<=new_inst.max_cycle_length)
    # Chaque sous-problème doit représenter un cycle
    #@constraint(SP,sum(x[i,l] for i in ((l+1):nv(new_inst.graph)) if (new_inst.edge_weight[i,l]==1.0))==1)
    @constraint(SP,sum(x[i,l] for i in Vl if (Al[i,l]==1))==1)
    #@constraint(SP,[j in (l+1):nv(new_inst.graph)],sum(x[i,j] for i in l:nv(new_inst.graph) if (new_inst.edge_weight[i,j]==1.0))<=1)
    @constraint(SP,[j in Vl],sum(x[i,j] for i in Vl if (Al[i,j]==1 && j!=l))<=1)
    #@constraint(SP,[i in l:nv(new_inst.graph)],sum(x[j,i] for j in l:nv(new_inst.graph) if (new_inst.edge_weight[j,i]==1.0))==sum(x[i,j] for j in l:nv(new_inst.graph) if (new_inst.edge_weight[i,j]==1.0)))
    @constraint(SP,[i in Vl],sum(x[j,i] for j in Vl if (Al[j,i]==1))==sum(x[i,j] for j in 1:nv(new_inst.graph) if (Al[i,j]==1)))
    # Objectif
    #@objective(SP,Max,sum(x[i,j] for i in l:nv(new_inst.graph),j in l:nv(new_inst.graph) if (new_inst.edge_weight[i,j]==1.0))*(1-sum(π[i] for i in l:nv(new_inst.graph))))
    @objective(SP,Max,sum(x[i,j] for i in Vl,j in Vl if (Al[i,j]==1))-sum(π[j]*sum(x[i,j] for i in Vl if (Al[i,j]==1)) for j in Vl))
    
    for (i,j) in tree[node].setzero
        @constraint(SP,x[i,j]<=0)
    end
    
    for (i,j) in tree[node].setone
        @constraint(SP,x[i,j]>=1)
    end

    optimize!(SP)
    
    print("statut",JuMP.termination_status(SP),"\n")
    
    # Si le sous-problème est résolu à l'optimalité, on retourne l'objectif et le motif optimal
    if JuMP.termination_status(SP)==MOI.OPTIMAL
        solution=Vector{Int}()
        for i in 1:nv(new_inst.graph)
            val=0
            for j in 1:nv(new_inst.graph)
                val=val+JuMP.value.(x)[i,j]
            end
            push!(solution,val)
        end
        return JuMP.objective_value(SP),solution
    # Sinon, le sous-problème est infaisable et on retourne Inf, []
    else
        return Inf,[]
    end
end

LoadError: LoadError: UndefVarError: @variable not defined
in expression starting at In[6]:20

In [5]:
function mindist(dist, sptset,graph)
    min = Inf  # Initialize minimum distance for next node
    minindex = 0
    # Search smallest value nearest vertex not in the
    # shortest path tree
    for i in 1:size(graph)[1]
        if dist[i] < min && sptset[i] == false
            min = dist[i]
            minindex = i
        end
    end
    return minindex
end

mindist (generic function with 2 methods)

In [ ]:
function dijkstra(graph, initial_node)
    TreeSet = [false for i in 1:size(graph)[1]] # step 1
    dist = [Inf for i in 1:size(graph)[1]] # step 2
    dist[initial_node] = 0

    for i in 1:size(graph)[1]

        # Pick the minimum distance vertex from
        # the set of vertices not yet processed.
        x = mindist(dist,TreeSet,graph) # step 3
        
        if x!=0
            # step 3 -> relaxation procedure
            # Update dist value of the adjacent vertices
            # of the picked vertex only if the current
            # distance is greater than new distance and
            # the vertex in not in the shortest path tree
            for j in 1:size(graph)[1]
                if graph[x,j] > 0 && TreeSet[j] == false && dist[j] > dist[x] + graph[x,j]
                    dist[j] = dist[x] + graph[x,j]
                end
            end

            # Put the minimum distance vertex in the
            # shortest path tree
            TreeSet[x] = true # step 4
        end
    end
    listeb=[j for (i , j) in enumerate(dist)]
    return listeb
end